# 💡 MosaicPi_Guide: **"Linear Algebra in MosaicPi"**

> All code and examples are shared to help researchers, students, and engineers understand the reasoning behind DDDA — and to make it easy to apply dimensional analysis to your own data.  
> This notebook serves as an entry-level guide for teaching, validating physical models, and enabling domain-specific knowledge engineering through data-driven dimensional reasoning.

---

## 🎯 What You'll Learn

**MosaicPi中的线性代数应用**

This notebook introduces the **fundamental concepts of linear algebra** that are widely used in applied mathematics, data science, physics, and engineering. The focus is on building an intuitive understanding alongside practical computations using NumPy.

1. **物理模型，隐函数，流形**  
   Understand why we reduce variables and how dimensional consistency enables model generalization.

2. **变量组合**  
   Encode physical units of input quantities using base units and build the D-matrix.

3. **变量组合评估**  
   Discover dimensionless groups by solving linear algebraic equations on the D-matrix.

4. **显式化策略可视化**  
   Learn to assess whether derived groups make physical and computational sense.

5. **不确定性定量化**  
   Set the stage for further steps in the DDDA pipeline including Pi-group selection, uncertainty quantification, and regime detection.

---

## 👤 Author

- **Name**: Jiashun Pang  
- **Created**: August 2025  
- **Affiliation**: MosaicPi, open research notebook  
- **Notebook Focus**:  
  A hands-on exploration of dimensional analysis — from aggregated raw quantities to symbolic Pi-group discovery and preparation for downstream DDDA tasks.

---

📌 *This notebook is designed to be accessible for learners new to dimensional analysis, while also laying the foundation for advanced applications in the full MosaicPi pipeline.*

---

# 7. 数值稳定性与灵敏度（Stability & Sensitivity）

## 7.1 条件数 $\kappa(A)$

* **定义（2-范数）**：$\displaystyle \kappa_2(A)=\|A\|_2\,\|A^{-1}\|_2=\frac{\sigma_{\max}(A)}{\sigma_{\min}(A)}$（奇异值比）
* **解释**：$\kappa$ 越大，输入/系数的微小扰动在输出中被放大得越厉害；$\kappa=1$ 最稳健。
* **常见范数**：$\kappa_p(A)=\|A\|_p\,\|A^{-1}\|_p$；避免用正规方程 $A^\top A$（会把条件数平方）。

## 7.2 矩阵扰动理论（要点速览）

* **线性方程 $Ax=b$** 的相对误差一阶界：

  $$
  \frac{\|\delta x\|}{\|x\|}\ \lesssim\ \kappa(A)\left(\frac{\|\delta b\|}{\|b\|}+\frac{\|\delta A\|}{\|A\|}\right)\Big/\big(1-\kappa(A)\frac{\|\delta A\|}{\|A\|}\big)
  $$
* **奇异值扰动（Weyl）**：$|\sigma_i(A+\Delta)-\sigma_i(A)|\le \|\Delta\|$
* **特征值敏感性**

  * 正规/对称：稳健，$|\delta\lambda_i|\le \|\Delta\|$
  * 一般可对角化矩阵（Bauer–Fike）：$|\lambda'-\lambda|\le \kappa(V)\,\|\Delta\|$（$V$ 为特征向量矩阵）
  * **非正规性** → 伪谱敏感，微小扰动可显著移动谱
* **不变子空间角度（Davis–Kahan）**：度量特征子空间旋转的稳健性

## 7.3 误差放大效应（Error Amplification）

* **输入噪声 $\to$ 输出误差**：$\kappa$ 是放大因子上界
* **模型病态的信号**：$\sigma_{\min}$ 很小、$\kappa$ 很大、残差对正则化敏感
* **避免技巧**：SVD/QR 替代正规方程；缩放与无量纲化；必要时做岭/Tikhonov 或 TSVD

## 7.4 在隐函数求解中的作用

设隐函数 $F(x,y)=0$，线性化：$J_y\,\Delta y = -J_x\,\Delta x$

* **可逆性**：$\det(J_y)\neq 0$（或 $\sigma_{\min}(J_y)>0$）
* **灵敏度界**：

  $$
  \|\Delta y\|\ \le\ \|J_y^{-1}\|\,\|J_x\|\,\|\Delta x\| \quad\Rightarrow\quad
  \text{放大因子}\ \approx\ \kappa(J_y)\ \text{（同量纲缩放后）}
  $$
* **噪声/估计误差传播**（数据驱动 $\hat J$ 的情形）：

  $$
  \Delta y \approx -\hat J_y^{+}\hat J_x \Delta x,\quad 
  \mathrm{Var}(\Delta y)\approx \hat J_y^{+}\ \mathrm{Var}(F)\ (\hat J_y^{+})^\top
  $$
* **判定哪些变量组合稳健**：

  * 扫描候选 $(x,y)$ 拆分，计算 $\sigma_{\min}(J_y),\,\kappa(J_y)$
  * 优先选择 **$\kappa(J_y)$ 小、$\sigma_{\min}$ 大** 的组合（并考虑实验可控性）
* **显函数置信度评分（建议指标）**：

  $$
  \text{score}=\underbrace{\phi_1\!\left(\frac{1}{1+\kappa(J_y)}\right)}_{\text{稳定性}}
  \cdot
  \underbrace{\phi_2\!\left(\frac{\sigma_{\min}(J_y)}{\sigma_{\max}(J_y)}\right)}_{\text{可逆裕度}}
  \cdot
  \underbrace{\phi_3\!\left(\frac{1}{1+\text{CV-resid}}\right)}_{\text{拟合残差}}
  \cdot
  \underbrace{\phi_4\!\left(\frac{r_0}{r}\right)}_{\text{线性化半径}}
  $$

  * 其中 $\phi_k\in(0,1]$ 为可选映射（如 logistic），$r$ 为局部样本半径，$r_0$ 期望半径

## 7.5 实践流程（可落地）

1. **缩放/无量纲化**：统一尺度，降低 $\kappa$ 的人为放大
2. **数据采样设计**：正交/满秩小扰动，确保 $\Delta X$ 条件良好
3. **SVD 评估 $J_y$**：取 $\sigma_{\min},\ \kappa,$ 有效秩（阈值 $\tau$）
4. **稳定化**：若 $\kappa$ 大，使用岭/TSVD；报告所用正则参数
5. **灵敏度地图**：网格化扫描候选 $(x,y)$ 组合，画 $\kappa(J_y)$/$\sigma_{\min}$ 热力图
6. **置信度打分**：按 7.4 的 score 合成，并排序给出推荐组合
7. **验证**：交叉验证预测误差；Bootstrap 评估 score 置信区间；必要时收缩线性化半径

## 7.6 常见陷阱与对策

* **使用 $A^\top A$**：$\kappa$ 平方，病态恶化 → 用 QR/SVD
* **尺度不一致**：未标准化导致方向主导 → 先无量纲化/标准化
* **非正规矩阵**：谱对扰动超敏 → 关注伪谱，避免仅看特征值
* **过拟合正则**：$\lambda$ 过大偏差上升 → 用 L-curve/GCV 选 $\lambda$

## 7.7 快速公式卡

* $\kappa_2(A)=\sigma_{\max}/\sigma_{\min}$
* 线性化：$\Delta y\approx -J_y^{-1}J_x\Delta x$ 或 $-J_y^{+}J_x\Delta x$
* 误差放大上界：$\|\Delta y\|\lesssim \|J_y^{-1}\|\,\|J_x\|\,\|\Delta x\|$
* SVD 稳定化：岭滤波系数 $\sigma/(\sigma^2+\lambda^2)$，TSVD：保留 $\sigma_i\ge\tau$

---
